In [ ]:
import nixio
import time
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_formats = ['retina'] # only for users with a high resolution display 

# Exercise 1: Enabling compression in the "radar_trap" data with images.

1. Test for the effect of the compression  and compare the file sizes with and without compression.

## Your solution

# Exercise 2: store data with calibration

1. Use the example code from the tutorial and create the nix file. Close it properly.
2. Reopen the file.
3. Compare the ``original_data`` and the data re-read from the file by plotting them.
4. What is the file size when you use either method? Can compression alone help?

## Your solution

## Exercise 3: Let's test the effects of chunking on the write performance.

1. Use the code below and extend it to test different chunk sizes (chunk_samples controls how many samples per channel at a time). **Note:** make sure to have the same total number of samples written. A good total number of samples is about 1Mio, start with chunk_samples > 100.
2. Compare the write performance with and without compression.
3. Select the "optimal" chunking strategy and test the read performance with slices of varying size.

## Your solution


In [ ]:

def record_data(samples, channels, dt):
    data = np.zeros((samples, channels))
    t = np.arange(samples) * dt
    for i in range(channels):
        phase = i * 2 * np.pi / channels
        data[:, i] = np.sin(2 * np.pi * t + phase) + (np.random.randn(samples) * 0.1)

    return data


def write_nixfile(filename, chunk_samples=1000, number_of_channels= 10, dt=0.001, chunk_count=100, compression=nixio.Compression.No):
    nixfile = nixio.File.open(filename, nixio.FileMode.Overwrite, compression=compression)
    block = nixfile.create_block("Session 1", "nix.recording_session")
    data_array = block.create_data_array("multichannel_data", "nix.sampled.multichannel", dtype=nixio.DataType.Double,
                                         shape=(chunk_samples, number_of_channels), label="voltage", unit="mV")
    data_array.append_sampled_dimension(0.001, label="time", unit="s")
    data_array.append_set_dimension(labels=["channel %i" % i for i in range(number_of_channels)])
    
    total_samples = chunk_count * chunk_samples
    data = record_data(total_samples, number_of_channels, dt)
    chunks_recorded = 0
    t0 = time.time()
    while chunks_recorded < chunk_count:
        start_index = chunk_samples * chunks_recorded
        if chunks_recorded == 0:
            data_array.write_direct(data[start_index:start_index + chunk_samples, :])
        else:
            data_array.append(data[start_index:start_index+chunk_samples, :], axis=0)
        chunks_recorded += 1
    total_time = time.time() - t0

    nixfile.close()
    return total_time


time_needed = write_nixfile("chunking_test.nix", chunk_samples=100000, chunk_count=10)
print(time_needed)